In [1]:
INPUT_CSV = r"C:\Users\82109\Desktop\벼리\7학기\텍스트마이닝\scripts\텍스트마이닝_초록_검사결과_전체컬럼1618.csv"
INPUT_XLSX = INPUT_CSV




In [2]:
OUTDIR = r"C:\Users\82109\Desktop\벼리\7학기\텍스트마이닝\방법론2\TF-IDF1618"





In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import re, os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer





In [4]:
# 분석할 '그 파일'의 정확한 경로를 직접 넣으세요
INPUT_CSV = r"C:\Users\82109\Desktop\벼리\7학기\텍스트마이닝\scripts\텍스트마이닝_초록_검사결과_전체컬럼1618.csv"
TITLE_COL  = "제목"     # 없으면 None
ABS_COL = "abstract"

# ===== TF-IDF 옵션 =====
USE_BIGRAMS = True   # (1,2) n-gram 사용할지
MIN_DF      = 2      # 너무 희귀한 토큰 제거(문서 수 기준)
MAX_DF      = 0.95   # 너무 흔한 토큰 제거(문서 비율 기준)








In [5]:
# 한글/영문/숫자 토큰 추출용 정규식 (길이 >= 2)
# 예: "부상 예방", "soccer", "2019" → 토큰
_TOKEN_RE = re.compile(r"[가-힣]{2,}|[A-Za-z]{2,}|[0-9]{2,}")

# 필요시 도메인 불용어(자유롭게 추가/수정)
STOPWORDS = {
    "연구", "결과", "방법", "분석", "본", "최근", "활용", "제시", "문헌",
    "the", "and", "for", "with", "from", "using", "based", "study", "results"
}

def normalize_text(s: str) -> str:
    if not isinstance(s, str): return ""
    s = s.lower()
    # 도메인 규칙: fifa 11+ → fifa11plus 로 묶어주기
    s = re.sub(r"fifa\s*11\+?", "fifa11plus", s, flags=re.I)
    # URL/특수기호 정리
    s = re.sub(r"https?://\S+|www\.\S+", " ", s)
    s = re.sub(r"[\u200b-\u200d\ufeff]", " ", s)
    s = re.sub(r"[^\w\s가-힣+]", " ", s)   # +는 11+ 같은 표기를 위해 남겨둠
    s = re.sub(r"\s+", " ", s).strip()
    return s

# “문자(한글/영문)가 최소 1개 포함된” 토큰만 추출
_TOKEN_RE = re.compile(r"(?=[\w+]*[가-힣A-Za-z])[가-힣A-Za-z0-9+]{2,}")


def regex_tokenize(text: str):
    text = normalize_text(text)
    toks = _TOKEN_RE.findall(text)
    # 숫자만으로 구성된 토큰 제거(이중 안전장치)
    toks = [t for t in toks if not t.isdigit()]
    # 불용어 제거
    toks = [t for t in toks if t not in STOPWORDS]
    return toks





In [6]:
# 1) 데이터 로드
assert os.path.exists(INPUT_CSV), f"파일이 없습니다: {INPUT_XLSX}"
df = pd.read_csv(INPUT_CSV)

assert ABS_COL in df.columns, f"'{ABS_COL}' 컬럼을 찾을 수 없습니다."
texts = df[ABS_COL].astype(str).tolist()

# 2) TF-IDF 빌드 (정규식 토크나이저 사용)
ngram = (1, 2) if USE_BIGRAMS else (1, 1)
vectorizer = TfidfVectorizer(
    tokenizer=regex_tokenize,   # 커스텀 토크나이저
    token_pattern=None,         # <- 커스텀 tokenizer 쓸 때 반드시 None
    ngram_range=ngram,
    min_df=MIN_DF,
    max_df=MAX_DF,
    stop_words=list(STOPWORDS)
)
X = vectorizer.fit_transform(texts)               # (문서수, 용어수) CSR 희소행렬
terms = np.array(vectorizer.get_feature_names_out())

print("TF-IDF shape:", X.shape)
print("예시 terms:", terms[:20])






TF-IDF shape: (765, 20019)
예시 terms: ['10g' '1rm' '1rm was' '1st' '20g' '2c' '2d' '2m' '2n' '2nd' '3d'
 '3d kinematic' '3d kinematics' '3d motion' '3d video' '3rd' '3x'
 '3x week' '40g' '50th']


상위 키워드 5개씩 뽑기 - 논문당

In [7]:
import numpy as np
import pandas as pd

TOPK_PER_DOC = 5  # 문서당 추출할 키워드 개수

def topk_keywords_per_doc(X, terms, k=5):
    """각 문서의 TF-IDF 상위 k개 (term, score) 리스트 반환"""
    results = []
    for i in range(X.shape[0]):
        row = X.getrow(i)
        if row.nnz == 0:
            results.append([])
            continue
        # 상위 k개 인덱스 (k보다 작으면 가능한 만큼)
        take = min(k, row.nnz)
        part = np.argpartition(row.data, -take)[-take:]      # 상위 k개 *무정렬*
        order = part[np.argsort(row.data[part])[::-1]]       # TF-IDF 내림차순 정렬
        term_idx = row.indices[order]
        scores = row.data[order]
        results.append(list(zip(terms[term_idx], scores)))
    return results

# 1) 문서별 상위 5 키워드 (term, score) 추출
topk = topk_keywords_per_doc(X, terms, k=TOPK_PER_DOC)

# 2) 결과 DataFrame 구성: 제목 + 상위키워드(문자열)
title = df['title'] if "title" in df.columns else pd.Series([""] * len(df))
date = df["date"] if "date" in df.columns else pd.Series([""] * len(df))
year = df["year"] if "year" in df.columns else pd.Series([""] * len(df))
topk_str = [", ".join([f"{t}({s:.3f})" for t, s in pairs]) for pairs in topk]

# (옵션) 개별 컬럼으로도 뽑기: kw1..kw5, score1..score5
def pad(lst, n, fill=None):  # 길이가 n보다 짧으면 채워 넣기
    return lst + [fill] * (n - len(lst))

kw_cols = []
sc_cols = []
for pairs in topk:
    terms_only  = pad([t for t, _ in pairs], TOPK_PER_DOC, "")
    scores_only = pad([f"{s:.3f}" for _, s in pairs], TOPK_PER_DOC, "")
    kw_cols.append(terms_only)
    sc_cols.append(scores_only)

kw_df = pd.DataFrame(kw_cols, columns=[f"kw{i+1}" for i in range(TOPK_PER_DOC)])
sc_df = pd.DataFrame(sc_cols, columns=[f"score{i+1}" for i in range(TOPK_PER_DOC)])

result_df = pd.DataFrame({"title": title, "date": date,"year": year,"top5_keywords": topk_str})
result_df = pd.concat([result_df, kw_df, sc_df], axis=1)

# 3) 저장
out_path = os.path.join(OUTDIR, f"tfidf_top{TOPK_PER_DOC}_per_doc.csv")
result_df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("[save]", out_path)

# 4) 샘플 확인 (앞 5개)
result_df.head()






[save] C:\Users\82109\Desktop\벼리\7학기\텍스트마이닝\방법론2\TF-IDF1618\tfidf_top5_per_doc.csv


,title,date,year,top5_keywords,kw1,kw2,kw3,kw4,kw5,score1,score2,score3,score4,score5
0,The VEGFA gene and anterior cruciate ligament ...,2018-12-28,2018,"aclr(0.463), aclr in(0.171), allele(0.164), gr...",aclr,aclr in,allele,group consisted,group,0.463,0.171,0.164,0.164,0.162
1,The relationship between ACTN3 R577X gene poly...,2018-12-28,2018,"genotyped(0.308), distribution(0.283), polymor...",genotyped,distribution,polymorphism,arm swing,arm,0.308,0.283,0.213,0.207,0.177
2,Reliability and validity of field‐based fitnes...,2018-12-27,2018,"fitness tests(0.254), tests(0.236), fitness(0....",fitness tests,tests,fitness,standards,field,0.254,0.236,0.215,0.183,0.172
3,A mathematical model for decision-making in th...,2018-12-24,2018,"classification(0.306), impairment(0.208), pals...",classification,impairment,palsy,cerebral palsy,para,0.306,0.208,0.176,0.176,0.168
4,Postural Control Deficits After Repetitive Soc...,2018-12-15,2018,"heading(0.245), soccer heading(0.232), postura...",heading,soccer heading,postural control,control,postural,0.245,0.232,0.226,0.211,0.208


이제, 이 논문들의 키워드를 가지고 클러스터링을 진행함

In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
assert "year" in df.columns, " df에 'year' 컬럼이 있어야 합니다."
assert "title" in df.columns, " df에 'title' 컬럼이 있어야 합니다."

YEAR_TARGET = 2018         
TITLE_COL   = "title"
DATE_COL    = "date"
YEAR_COL    = "year"

OUTDIR = "./TFIDF2018" # 여기 연도별로 바꾸기
os.makedirs(OUTDIR, exist_ok=True)

mask_year = (df[YEAR_COL] == YEAR_TARGET)
df_year = df.loc[mask_year].reset_index(drop=True)
if df_year.empty:
    raise ValueError(f"{YEAR_TARGET}년에 해당하는 문서가 없습니다.")

idx_year = np.where(mask_year.values)[0]
topk_year = [topk[i] for i in idx_year]
# ===============================
# 1) 문서별 top-k 결과(topk)를 "길게" 풀기
#    topk: List[List[(term, score)]]
# ===============================
rows = []
for i, pairs in enumerate(topk_year):
    title = df[TITLE_COL].iloc[i] if TITLE_COL in df.columns else i
    for term, val in pairs:
        rows.append({
            "year": YEAR_TARGET,
            "doc_id": i,
            "title": title,
            "keyword": term,
            "tfidf": float(val)
        })

kw_long = pd.DataFrame(rows)
print("[info] pooled keywords:", kw_long.shape, "unique:", kw_long["keyword"].nunique())
kw_long_path = os.path.join(OUTDIR, "pooled_keywords_top5_long.csv")
kw_long.to_csv(kw_long_path, index=False, encoding="utf-8-sig")
print("[save]", kw_long_path)

# ===============================
# 2) 키워드 자체를 벡터화
#    - 키워드가 짧으니 char n-gram이 안정적
#    - 공백이 있는 키워드(예: 'injury prevention')도 잘 커버
# ===============================
vectorizer_kw = TfidfVectorizer(
    analyzer="char_wb",       # 단어 경계 기반 문자 n-gram
    ngram_range=(3,5),        # 3~5그램 권장(짧은 토큰에 강함)
    min_df=1
)
X_kw = vectorizer_kw.fit_transform(kw_long["keyword"])
print("X_kw shape:", X_kw.shape)

# ===============================
# 3) KMeans 클러스터링 (k는 필요에 따라 조절)
#    - 실루엣으로 후보 k 점검 후 최종 k 선택
# ===============================
k_candidates = [4,5,6,7,8]
scores = []
for k in k_candidates:
    if X_kw.shape[0] > k:
        km_tmp = KMeans(n_clusters=k, n_init="auto", random_state=42)
        labels_tmp = km_tmp.fit_predict(X_kw)
        scores.append(silhouette_score(X_kw, labels_tmp))
    else:
        scores.append(np.nan)

sil_df = pd.DataFrame({"k": k_candidates, "silhouette": scores})
print(sil_df)

# 최종 k 선택(직접 지정해도 됨)
K_CLUSTERS = int(sil_df.iloc[sil_df["silhouette"].idxmax()]["k"]) if sil_df["silhouette"].notna().any() else 6
print("[info] choose k =", K_CLUSTERS)

km = KMeans(n_clusters=K_CLUSTERS, n_init="auto", random_state=42)
kw_long["cluster"] = km.fit_predict(X_kw)

clusters_csv = os.path.join(OUTDIR, f"{YEAR_TARGET}keyword_clusters_k{K_CLUSTERS}.csv")
kw_long.to_csv(clusters_csv, index=False, encoding="utf-8-sig")
print("[save]", clusters_csv)

# ===============================
# 4) 클러스터 요약 (빈도/대표 키워드/대표 문서)
# ===============================
# (a) 클러스터별 키워드 빈도 상위 샘플
top_kw_by_cluster = (
    kw_long.groupby(["cluster","keyword"])
           .size()
           .reset_index(name="freq")
           .sort_values(["cluster","freq"], ascending=[True, False])
)

summary_rows = []
for c in sorted(kw_long["cluster"].unique()):
    sample = top_kw_by_cluster[top_kw_by_cluster["cluster"]==c].head(15)["keyword"].tolist()
    summary_rows.append({"cluster": c, "sample_keywords": ", ".join(sample)})
summary_df = pd.DataFrame(summary_rows)
summary_csv = os.path.join(OUTDIR, f"{YEAR_TARGET}keyword_cluster_summary_k{K_CLUSTERS}.csv")
summary_df.to_csv(summary_csv, index=False, encoding="utf-8-sig")
print("[save]", summary_csv)

# (b) 클러스터별 대표 문서(해당 클러스터에 속한 키워드의 tf-idf 합 기준 상위)
doc_rank = (
    kw_long.groupby(["cluster","title"])["tfidf"]
           .sum()
           .reset_index()
           .sort_values(["cluster","tfidf"], ascending=[True, False])
)
top_docs = doc_rank.groupby("cluster").head(10)
top_docs_csv = os.path.join(OUTDIR, f"{YEAR_TARGET}keyword_cluster_top_docs_k{K_CLUSTERS}.csv")
top_docs.to_csv(top_docs_csv, index=False, encoding="utf-8-sig")
print("[save]", top_docs_csv)

# ===============================
# 5) 화면에 간단 프린트
# ===============================
for c in sorted(kw_long["cluster"].unique()):
    sample_kw = top_kw_by_cluster[top_kw_by_cluster["cluster"]==c].head(10)["keyword"].tolist()
    print(f"\n=== Cluster {c} ===")
    print(", ".join(sample_kw))






[info] pooled keywords: (1420, 5) unique: 1121
[save] ./TFIDF2018\pooled_keywords_top5_long.csv
X_kw shape: (1420, 7922)
   k  silhouette
0  4    0.006440
1  5    0.011522
2  6    0.012511
3  7    0.013660
4  8    0.011868
[info] choose k = 7
[save] ./TFIDF2018\2018keyword_clusters_k7.csv
[save] ./TFIDF2018\2018keyword_cluster_summary_k7.csv
[save] ./TFIDF2018\2018keyword_cluster_top_docs_k7.csv

=== Cluster 0 ===
gps, lps, greater quadriceps, groups, jumps, mps, ps, quadriceps, quadriceps hamstrings, to quadriceps

=== Cluster 1 ===
pain, game, groin pain, intervention, programme, asymmetry, athlete, back pain, filter, an outer

=== Cluster 2 ===
classification, fitness, test, cutting, estimation, mechanical, mechanism, neck, physical fitness, rates

=== Cluster 3 ===
acl, concussion, ball, force, h3, head, hip, cod, exposure, strength

=== Cluster 4 ===
injuries, extremity injury, groin injuries, injury, of injury, sports injuries, acl injured, acl injuries, acl injury, ankle injurie